<a href="https://colab.research.google.com/github/DLNinja/EmotionResNET/blob/main/EEGEmotionResnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import re
from tqdm import tqdm

DATASET_PATH_STFT = "/content/drive/MyDrive/Datasets/SEED/de_stft.npz"
DATASET_PATH_BANDPASS = "/content/drive/MyDrive/Datasets/SEED/de_bandpass.npz"
DATASET_PATH_STFT_SMOOTH = "/content/drive/MyDrive/Datasets/SEED/de_stft_smooth.npz"
DATASET_PATH_BANDPASS_SMOOTH = "/content/drive/MyDrive/Datasets/SEED/de_bandpass_smooth.npz"

# Dataset and Model classes

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset

class SEEDDataset(Dataset):
    def __init__(self, npz_path, person_ids=None, transform=None):
        data = np.load(npz_path)
        X = data['X']
        y = data['y']
        persons = data['persons']
        sessions = data['sessions']

        if person_ids is not None:
            mask = np.isin(persons, person_ids)
            X = X[mask]
            y = y[mask]
            persons = persons[mask]
            sessions = sessions[mask]

        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
        self.y = torch.tensor(y, dtype=torch.long)
        self.persons = torch.tensor(persons, dtype=torch.long)
        self.sessions = torch.tensor(sessions, dtype=torch.long)
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.y[idx]
        if self.transform:
            x = self.transform(x)
        return x, y, self.persons[idx], self.sessions[idx]

In [ ]:
train_persons = np.arange(0, 13)
test_persons = np.arange(13, 15)

train_dataset = SEEDDataset(DATASET_PATH_STFT_SMOOTH, person_ids=train_persons)
test_dataset = SEEDDataset(DATASET_PATH_STFT_SMOOTH, person_ids=test_persons)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Train samples:", len(train_dataset))
print("Test samples:", len(test_dataset))


Train samples: 131599
Test samples: 20246


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class EEGResNet(nn.Module):
    def __init__(self, num_classes=4, pretrained=True):
        super(EEGResNet, self).__init__()

        self.backbone = models.resnet18(pretrained=pretrained)

        self.backbone.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=64,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False
        )

        self.backbone.maxpool = nn.Identity()

        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.backbone(x)


# Training process

## Subject-Dependent Training
Model is trained on data from all persons, and from each session

In [ ]:
full_data = np.load(DATASET_PATH_BANDPASS_SMOOTH)

y = full_data["y"]
persons = full_data["persons"]
sessions = full_data["sessions"]

N = len(y)
indices = np.arange(N)

# Create joint stratification key
# Each unique (person, session, label) gets preserved
stratify_key = np.array([
    f"{p}_{s}_{l}" for p, s, l in zip(persons, sessions, y)
])

train_idx, test_idx = train_test_split(
    indices,
    test_size=0.20,
    shuffle=True,
    random_state=42,
    stratify=stratify_key
)

print(f"Train size: {len(train_idx)}")
print(f"Test  size: {len(test_idx)}")

full_dataset = SEEDDataset(DATASET_PATH_BANDPASS_SMOOTH, person_ids=None)

train_dataset = Subset(full_dataset, train_idx)
test_dataset  = Subset(full_dataset, test_idx)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
model = EEGResNet(num_classes=num_classes, pretrained=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-2
)

metrics = {
    "train_acc": [],
    "train_loss": [],
    "val_acc": [],
    "val_loss": []
}

best_val_acc = 0.0
for epoch in range(1, num_epochs + 1):
    print(f"\n=== Epoch {epoch}/{num_epochs} ===")

    # -------- TRAIN --------
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    loop = tqdm(train_loader, desc="Training")

    for x, y, _, _ in loop:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        _, pred = out.max(1)
        total += y.size(0)
        correct += pred.eq(y).sum().item()

        loop.set_postfix(
            loss=train_loss / total,
            acc=100 * correct / total
        )

    train_acc = 100 * correct / total
    train_loss /= total

    # -------- VALIDATION --------
    model.eval()
    val_loss, cor, tot = 0.0, 0, 0

    with torch.no_grad():
        for x, y, _, _ in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)

            val_loss += loss.item() * x.size(0)
            _, pred = out.max(1)
            tot += y.size(0)
            cor += pred.eq(y).sum().item()

    val_acc = 100 * cor / tot
    val_loss /= tot

    metrics["train_acc"].append(train_acc)
    metrics["train_loss"].append(train_loss)
    metrics["val_acc"].append(val_acc)
    metrics["val_loss"].append(val_loss)

    np.save(METRICS_SAVE_PATH, metrics)

    print(
        f"Train Acc: {train_acc:.2f}% | "
        f"Val Acc: {val_acc:.2f}%"
    )

    # -------- Save best model --------
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(
            {
                "epoch": epoch,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "val_acc": val_acc
            },
            BEST_MODEL_PATH
        )
        print(f"Best model saved (Val Acc = {val_acc:.2f}%)")

## Leave One Subject Out (LOSO)
Model trained on 13 persons, then tested on two persons it hasn't seen

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

num_epochs = 50
batch_size = 32
num_classes = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_persons = [0, 1]     # Persons evaluated in parallel
metrics = {p: {"val_acc": [], "val_loss": []} for p in test_persons}
METRICS_SAVE_PATH = "/content/drive/MyDrive/Datasets/SEED/ResnetMetrics/two_person_metrics_stft.npz"

full_data = np.load(DATASET_PATH_STFT_SMOOTH)
unique_persons = np.unique(full_data["persons"])

train_persons = [p for p in unique_persons if p not in test_persons]

train_dataset = SEEDDataset(DATASET_PATH_STFT_SMOOTH, person_ids=train_persons)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_datasets = {
    p: DataLoader(
        SEEDDataset(DATASET_PATH_STFT_SMOOTH, person_ids=[p]),
        batch_size=batch_size,
        shuffle=False
    )
    for p in test_persons
}

model = EEGResNet(num_classes=num_classes, pretrained=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

for epoch in range(1, num_epochs + 1):
    print(f"\n=== Epoch {epoch}/{num_epochs} ===")

    # -------- TRAIN --------
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    loop = tqdm(train_loader, desc="Training")

    for x, y, _, _ in loop:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        _, pred = out.max(1)
        total += y.size(0)
        correct += pred.eq(y).sum().item()

        loop.set_postfix(loss=train_loss/total, acc=100*correct/total)

    print(f"Train Acc = {100*correct/total:.2f}%  |  Loss = {train_loss/total:.4f}")

    # -------- VALIDATION --------
    model.eval()

    for person_id in test_persons:
        tot, cor, val_loss = 0, 0, 0.0
        loader = test_datasets[person_id]

        with torch.no_grad():
            for x, y, _, _ in loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                loss = criterion(out, y)

                val_loss += loss.item() * x.size(0)
                _, pred = out.max(1)
                tot += y.size(0)
                cor += pred.eq(y).sum().item()

        person_acc = 100 * cor / tot
        person_loss = val_loss / tot

        metrics[person_id]["val_acc"].append(person_acc)
        metrics[person_id]["val_loss"].append(person_loss)

        print(f" Person {person_id} → Val Acc: {person_acc:.2f}% | Val Loss: {person_loss:.4f}")

    np.save(METRICS_SAVE_PATH, metrics)

print("\nTraining Finished!")
print("Final Metrics:", metrics)


## %5 test Person Data
Model trains on 13 subjects and 5% from the two test persons

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
from sklearn.model_selection import train_test_split

num_epochs = 50
batch_size = 32
num_classes = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_persons = [0, 1]
metrics = {p: {"val_acc": [], "val_loss": []} for p in test_persons}

METRICS_SAVE_PATH = "/content/drive/MyDrive/Datasets/SEED/two_person_5_training_metrics.npz"

full_data = np.load(DATASET_PATH_STFT_SMOOTH)
persons_arr = full_data["persons"]
unique_persons = np.unique(persons_arr)

train_indices = []
test_indices = []

for person_id in test_persons:
    idx = np.where(persons_arr == person_id)[0]

    train_idx, test_idx = train_test_split(
        idx,
        test_size=0.95,    # 95% into test
        shuffle=True,
        random_state=42
    )

    train_indices.extend(train_idx)
    test_indices.extend(test_idx)

other_persons = [p for p in unique_persons if p not in test_persons]

for p in other_persons:
    idx = np.where(persons_arr == p)[0]
    train_indices.extend(idx)

train_indices = np.array(train_indices)
test_indices = np.array(test_indices)

print(f"Train samples from all persons: {len(train_indices)}")
print(f"Test  samples from p0+p1 (95%): {len(test_indices)}")

full_dataset = SEEDDataset(DATASET_PATH_STFT_SMOOTH, person_ids=None)

train_dataset = Subset(full_dataset, train_indices)
test_dataset_p0 = Subset(full_dataset, np.where((persons_arr == 0) & np.isin(np.arange(len(persons_arr)), test_indices))[0])
test_dataset_p1 = Subset(full_dataset, np.where((persons_arr == 1) & np.isin(np.arange(len(persons_arr)), test_indices))[0])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loaders = {
    0: DataLoader(test_dataset_p0, batch_size=batch_size, shuffle=False),
    1: DataLoader(test_dataset_p1, batch_size=batch_size, shuffle=False)
}

model = EEGResNet(num_classes=num_classes, pretrained=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

for epoch in range(1, num_epochs + 1):
    print(f"\n=== Epoch {epoch}/{num_epochs} ===")

    # -------- TRAIN ----------
    model.train()
    train_loss, total, correct = 0.0, 0, 0
    loop = tqdm(train_loader, desc="Training")

    for x, y, _, _ in loop:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        _, pred = out.max(1)
        total += y.size(0)
        correct += pred.eq(y).sum().item()

        loop.set_postfix(loss=train_loss/total, acc=100*correct/total)

    print(f"Train Acc = {100*correct/total:.2f}%  |  Loss = {train_loss/total:.4f}")

    # -------- VALIDATION (person 0 and person 1 separately) ----------
    model.eval()

    for pid in test_persons:
        loader = test_loaders[pid]
        tot, cor, vloss = 0, 0, 0.0

        with torch.no_grad():
            for x, y, _, _ in loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                loss = criterion(out, y)

                vloss += loss.item() * x.size(0)
                _, pred = out.max(1)
                tot += y.size(0)
                cor += pred.eq(y).sum().item()

        acc = 100 * cor / tot
        loss_val = vloss / tot

        metrics[pid]["val_acc"].append(acc)
        metrics[pid]["val_loss"].append(loss_val)

        print(f" Person {pid}: Val Acc = {acc:.2f}% | Loss = {loss_val:.4f}")

    np.save(METRICS_SAVE_PATH, metrics)

print("\nTraining done!")
print(metrics)


## %10 test Person Data
Model trains on 13 subjects and 10% from the two test persons

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
from sklearn.model_selection import train_test_split

num_epochs = 50
batch_size = 32
num_classes = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_persons = [0, 1]
metrics = {p: {"val_acc": [], "val_loss": []} for p in test_persons}

METRICS_SAVE_PATH = "/content/drive/MyDrive/Datasets/SEED/two_person_10_training_metrics.npz"

full_data = np.load(DATASET_PATH_STFT_SMOOTH)
persons_arr = full_data["persons"]
unique_persons = np.unique(persons_arr)

train_indices = []
test_indices = []

for person_id in test_persons:
    idx = np.where(persons_arr == person_id)[0]

    train_idx, test_idx = train_test_split(
        idx,
        test_size=0.90,    # 90% into test
        shuffle=True,
        random_state=42
    )

    train_indices.extend(train_idx)
    test_indices.extend(test_idx)

other_persons = [p for p in unique_persons if p not in test_persons]

for p in other_persons:
    idx = np.where(persons_arr == p)[0]
    train_indices.extend(idx)

train_indices = np.array(train_indices)
test_indices = np.array(test_indices)

print(f"Train samples from all persons: {len(train_indices)}")
print(f"Test  samples from p0+p1 (90%): {len(test_indices)}")

full_dataset = SEEDDataset(DATASET_PATH_STFT_SMOOTH, person_ids=None)

train_dataset = Subset(full_dataset, train_indices)
test_dataset_p0 = Subset(full_dataset, np.where((persons_arr == 0) & np.isin(np.arange(len(persons_arr)), test_indices))[0])
test_dataset_p1 = Subset(full_dataset, np.where((persons_arr == 1) & np.isin(np.arange(len(persons_arr)), test_indices))[0])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loaders = {
    0: DataLoader(test_dataset_p0, batch_size=batch_size, shuffle=False),
    1: DataLoader(test_dataset_p1, batch_size=batch_size, shuffle=False)
}

model = EEGResNet(num_classes=num_classes, pretrained=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

for epoch in range(1, num_epochs + 1):
    print(f"\n=== Epoch {epoch}/{num_epochs} ===")

    # -------- TRAIN ----------
    model.train()
    train_loss, total, correct = 0.0, 0, 0
    loop = tqdm(train_loader, desc="Training")

    for x, y, _, _ in loop:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        _, pred = out.max(1)
        total += y.size(0)
        correct += pred.eq(y).sum().item()

        loop.set_postfix(loss=train_loss/total, acc=100*correct/total)

    print(f"Train Acc = {100*correct/total:.2f}%  |  Loss = {train_loss/total:.4f}")

    # -------- VALIDATION (person 0 and person 1 separately) ----------
    model.eval()

    for pid in test_persons:
        loader = test_loaders[pid]
        tot, cor, vloss = 0, 0, 0.0

        with torch.no_grad():
            for x, y, _, _ in loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                loss = criterion(out, y)

                vloss += loss.item() * x.size(0)
                _, pred = out.max(1)
                tot += y.size(0)
                cor += pred.eq(y).sum().item()

        acc = 100 * cor / tot
        loss_val = vloss / tot

        metrics[pid]["val_acc"].append(acc)
        metrics[pid]["val_loss"].append(loss_val)

        print(f" Person {pid}: Val Acc = {acc:.2f}% | Loss = {loss_val:.4f}")

    np.save(METRICS_SAVE_PATH, metrics)

print("\nTraining done!")
print(metrics)

## %15 test Person Data
Model trains on 13 subjects and 15% from the two test persons

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
from sklearn.model_selection import train_test_split

num_epochs = 50
batch_size = 32
num_classes = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_persons = [0, 1]
metrics = {p: {"val_acc": [], "val_loss": []} for p in test_persons}

METRICS_SAVE_PATH = "/content/drive/MyDrive/Datasets/SEED/two_person_15_training_metrics.npz"

full_data = np.load(DATASET_PATH_STFT_SMOOTH)
persons_arr = full_data["persons"]
unique_persons = np.unique(persons_arr)

train_indices = []
test_indices = []

for person_id in test_persons:
    idx = np.where(persons_arr == person_id)[0]

    train_idx, test_idx = train_test_split(
        idx,
        test_size=0.85,    # 85% into test
        shuffle=True,
        random_state=42
    )

    train_indices.extend(train_idx)
    test_indices.extend(test_idx)

other_persons = [p for p in unique_persons if p not in test_persons]

for p in other_persons:
    idx = np.where(persons_arr == p)[0]
    train_indices.extend(idx)

train_indices = np.array(train_indices)
test_indices = np.array(test_indices)

print(f"Train samples from all persons: {len(train_indices)}")
print(f"Test  samples from p0+p1 (85%): {len(test_indices)}")

full_dataset = SEEDDataset(DATASET_PATH_STFT_SMOOTH, person_ids=None)

train_dataset = Subset(full_dataset, train_indices)
test_dataset_p0 = Subset(full_dataset, np.where((persons_arr == 0) & np.isin(np.arange(len(persons_arr)), test_indices))[0])
test_dataset_p1 = Subset(full_dataset, np.where((persons_arr == 1) & np.isin(np.arange(len(persons_arr)), test_indices))[0])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loaders = {
    0: DataLoader(test_dataset_p0, batch_size=batch_size, shuffle=False),
    1: DataLoader(test_dataset_p1, batch_size=batch_size, shuffle=False)
}

model = EEGResNet(num_classes=num_classes, pretrained=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

for epoch in range(1, num_epochs + 1):
    print(f"\n=== Epoch {epoch}/{num_epochs} ===")

    # -------- TRAIN ----------
    model.train()
    train_loss, total, correct = 0.0, 0, 0
    loop = tqdm(train_loader, desc="Training")

    for x, y, _, _ in loop:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        _, pred = out.max(1)
        total += y.size(0)
        correct += pred.eq(y).sum().item()

        loop.set_postfix(loss=train_loss/total, acc=100*correct/total)

    print(f"Train Acc = {100*correct/total:.2f}%  |  Loss = {train_loss/total:.4f}")

    # -------- VALIDATION (person 0 and person 1 separately) ----------
    model.eval()

    for pid in test_persons:
        loader = test_loaders[pid]
        tot, cor, vloss = 0, 0, 0.0

        with torch.no_grad():
            for x, y, _, _ in loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                loss = criterion(out, y)

                vloss += loss.item() * x.size(0)
                _, pred = out.max(1)
                tot += y.size(0)
                cor += pred.eq(y).sum().item()

        acc = 100 * cor / tot
        loss_val = vloss / tot

        metrics[pid]["val_acc"].append(acc)
        metrics[pid]["val_loss"].append(loss_val)

        print(f" Person {pid}: Val Acc = {acc:.2f}% | Loss = {loss_val:.4f}")

    np.save(METRICS_SAVE_PATH, metrics)

print("\nTraining done!")
print(metrics)


## Session based training

In [ ]:
full_data = np.load(DATASET_PATH_STFT_SMOOTH)
persons_arr  = full_data["persons"]
sessions_arr = full_data["sessions"]

TRAIN_SESSIONS = [0, 1]
TEST_SESSION   = 2

train_idx = np.where(np.isin(sessions_arr, TRAIN_SESSIONS))[0]
test_idx  = np.where(sessions_arr == TEST_SESSION)[0]

print(f"Train samples (sessions 0,1): {len(train_idx)}")
print(f"Test samples  (session 2)  : {len(test_idx)}")

test_persons = np.unique(persons_arr[test_idx]).tolist()
print("Test persons:", test_persons)

In [ ]:
full_dataset = SEEDDataset(DATASET_PATH_STFT_SMOOTH)

train_dataset = Subset(full_dataset, train_idx)

test_datasets = {
    pid: Subset(
        full_dataset,
        test_idx[persons_arr[test_idx] == pid]
    )
    for pid in test_persons
}

for pid in test_persons:
    print(f"Person {pid} → test samples: {len(test_datasets[pid])}")


In [ ]:
batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

test_loaders = {
    pid: DataLoader(
        test_datasets[pid],
        batch_size=batch_size,
        shuffle=False
    )
    for pid in test_persons
}

metrics = {
    "train": {
        "acc": [],
        "loss": []
    }
}

# Add per-person validation metrics
for pid in test_persons:
    metrics[pid] = {
        "val_acc": [],
        "val_loss": []
    }

In [ ]:
METRICS_SAVE_PATH = "/content/drive/MyDrive/Datasets/SEED/session_based_metrics_stft.npz"

num_epochs = 50
num_classes = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for epoch in range(1, num_epochs + 1):
    print(f"\n=== Epoch {epoch}/{num_epochs} ===")

    # -------- TRAIN --------
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    loop = tqdm(train_loader, desc="Training")

    for x, y, _, _ in loop:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        _, pred = out.max(1)
        total += y.size(0)
        correct += pred.eq(y).sum().item()

        loop.set_postfix(
            loss=train_loss / total,
            acc=100.0 * correct / total
        )

    train_acc = 100.0 * correct / total
    train_loss_avg = train_loss / total

    metrics["train"]["acc"].append(train_acc)
    metrics["train"]["loss"].append(train_loss_avg)

    print(f"Train Acc = {train_acc:.2f}% | "
          f"Loss = {train_loss_avg:.4f}")

    # -------- TEST (SESSION 2, ALL PERSONS) --------
    model.eval()

    for pid in test_persons:
        loader = test_loaders[pid]
        vloss, cor, tot = 0.0, 0, 0

        with torch.no_grad():
            for x, y, _, _ in loader:
                x, y = x.to(device), y.to(device)

                out = model(x)
                loss = criterion(out, y)

                vloss += loss.item() * x.size(0)
                _, pred = out.max(1)
                tot += y.size(0)
                cor += pred.eq(y).sum().item()

        acc = 100.0 * cor / tot if tot > 0 else 0.0
        loss_val = vloss / tot if tot > 0 else 0.0

        metrics[pid]["val_acc"].append(acc)
        metrics[pid]["val_loss"].append(loss_val)

        print(f" Person {pid:02d}: "
              f"Acc = {acc:6.2f}% | "
              f"Loss = {loss_val:.4f}")

    np.save(METRICS_SAVE_PATH, metrics)
